In [ ]:
import streamlit as st

In [ ]:
import sqlite3

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import plotly.express as px

In [ ]:
import plotly.graph_objects as go

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
import datetime

In [ ]:
from datetime import datetime, timedelta

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_database_connection():
    """Connect to Northwind SQLite database"""
    conn = sqlite3.connect('northwind.db', check_same_thread=False)
    return conn
conn = get_database_connection()
if conn:
    tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql_query(tables_query, conn)
    print("\n📋 Available tables in Northwind database:")
    for table in tables['name']:
        print(f"  - {table}")

In [ ]:
def get_sales_overview(conn):
    """Complex SQL query for sales overview with CTEs and window functions"""
    query = '''
    WITH monthly_sales AS (
        SELECT 
            strftime('%Y-%m', o.OrderDate) as order_month,
            strftime('%Y', o.OrderDate) as order_year,
            COUNT(DISTINCT o.OrderID) as total_orders,
            COUNT(DISTINCT o.CustomerID) as unique_customers,
            SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) as total_revenue,
            SUM(od.Quantity) as total_quantity,
            AVG(od.UnitPrice * od.Quantity * (1 - od.Discount)) as avg_order_value
        FROM Orders o
        JOIN [Order Details] od ON o.OrderID = od.OrderID
        WHERE o.OrderDate IS NOT NULL
        GROUP BY strftime('%Y-%m', o.OrderDate), strftime('%Y', o.OrderDate)
    ),
    monthly_growth AS (
        SELECT *,
            LAG(total_revenue) OVER (ORDER BY order_month) as prev_month_revenue,
            ((total_revenue - LAG(total_revenue) OVER (ORDER BY order_month)) / 
             NULLIF(LAG(total_revenue) OVER (ORDER BY order_month), 0)) * 100 as revenue_growth_rate,
            SUM(total_revenue) OVER (ORDER BY order_month ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_revenue
        FROM monthly_sales
    )
    SELECT * FROM monthly_growth ORDER BY order_month
    '''
    return pd.read_sql_query(query, conn)

def get_product_performance(conn):
    """Product performance analysis with ranking"""
    query = '''
    WITH product_sales AS (
        SELECT 
            p.ProductID,
            p.ProductName,
            c.CategoryName,
            s.CompanyName as SupplierName,
            SUM(od.Quantity) as total_quantity_sold,
            SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) as total_revenue,
            COUNT(DISTINCT od.OrderID) as order_count,
            AVG(od.UnitPrice * od.Quantity * (1 - od.Discount)) as avg_order_value,
            SUM(od.Discount * od.UnitPrice * od.Quantity) as total_discount_given
        FROM Products p
        JOIN Categories c ON p.CategoryID = c.CategoryID
        JOIN Suppliers s ON p.SupplierID = s.SupplierID
        JOIN [Order Details] od ON p.ProductID = od.ProductID
        GROUP BY p.ProductID, p.ProductName, c.CategoryName, s.CompanyName
    ),
    product_rankings AS (
        SELECT *,
            RANK() OVER (ORDER BY total_revenue DESC) as revenue_rank,
            RANK() OVER (ORDER BY total_quantity_sold DESC) as quantity_rank,
            RANK() OVER (PARTITION BY CategoryName ORDER BY total_revenue DESC) as category_rank
        FROM product_sales
    )
    SELECT * FROM product_rankings ORDER BY total_revenue DESC
    '''
    return pd.read_sql_query(query, conn)

def get_customer_analysis(conn):
    """Customer segmentation and cohort analysis"""
    query = '''
    WITH customer_metrics AS (
        SELECT 
            c.CustomerID,
            c.CompanyName,
            c.ContactName,
            c.City,
            c.Country,
            COUNT(DISTINCT o.OrderID) as order_count,
            SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) as total_revenue,
            AVG(od.UnitPrice * od.Quantity * (1 - od.Discount)) as avg_order_value,
            MIN(o.OrderDate) as first_order_date,
            MAX(o.OrderDate) as last_order_date,
            (julianday(MAX(o.OrderDate)) - julianday(MIN(o.OrderDate))) as customer_lifespan_days,
            SUM(od.Quantity) as total_items_purchased
        FROM Customers c
        JOIN Orders o ON c.CustomerID = o.CustomerID
        JOIN [Order Details] od ON o.OrderID = od.OrderID
        GROUP BY c.CustomerID, c.CompanyName, c.ContactName, c.City, c.Country
    ),
    customer_segments AS (
        SELECT *,
            CASE 
                WHEN total_revenue >= 10000 THEN 'High Value'
                WHEN total_revenue >= 5000 THEN 'Medium Value'
                ELSE 'Low Value'
            END as value_segment,
            CASE 
                WHEN order_count >= 15 THEN 'Frequent'
                WHEN order_count >= 8 THEN 'Regular'
                ELSE 'Occasional'
            END as frequency_segment,
            NTILE(5) OVER (ORDER BY total_revenue DESC) as revenue_quintile
        FROM customer_metrics
    )
    SELECT * FROM customer_segments ORDER BY total_revenue DESC
    '''
    return pd.read_sql_query(query, conn)

def get_employee_performance(conn):
    """Employee sales performance analysis"""
    query = '''
    SELECT 
        e.EmployeeID,
        e.FirstName || ' ' || e.LastName as EmployeeName,
        e.Title,
        COUNT(DISTINCT o.OrderID) as orders_handled,
        SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) as total_sales,
        AVG(od.UnitPrice * od.Quantity * (1 - od.Discount)) as avg_order_value,
        COUNT(DISTINCT o.CustomerID) as unique_customers_served
    FROM Employees e
    JOIN Orders o ON e.EmployeeID = o.EmployeeID
    JOIN [Order Details] od ON o.OrderID = od.OrderID
    GROUP BY e.EmployeeID, e.FirstName, e.LastName, e.Title
    ORDER BY total_sales DESC
    '''
    return pd.read_sql_query(query, conn)

print("All FIXED SQL query functions defined successfully!")


def test_fixed_queries():
    """Test the queries"""
    try:
        conn = sqlite3.connect('northwind.db', check_same_thread=False)
        
        print(" Testing  queries...")
        
        
        sales_data = get_sales_overview(conn)
        print(f" Sales Overview: {len(sales_data)} months loaded")
        
          
        product_data = get_product_performance(conn)
        print(f" Product Performance: {len(product_data)} products loaded")
        
        
        customer_data = get_customer_analysis(conn)
        print(f" Customer Analysis: {len(customer_data)} customers loaded")
        
        
        employee_data = get_employee_performance(conn)
        print(f" Employee Performance: {len(employee_data)} employees loaded")
        
        conn.close()
        
        print("\n ALL QUERIES WORKING.")
        
        return sales_data, product_data, customer_data, employee_data
        
    except Exception as e:
        print(f" Error: {e}")
        return None, None, None, None


test_fixed_queries()

In [ ]:
if conn:
    print("🔄 Loading data from Northwind database...")
    
    # Load all datasets
    sales_data = get_sales_overview(conn)
    product_data = get_product_performance(conn)
    customer_data = get_customer_analysis(conn)
    employee_data = get_employee_performance(conn)
    
    print(f" Sales Overview: {len(sales_data)} months of data")
    print(f" Product Performance: {len(product_data)} products")
    print(f" Customer Analysis: {len(customer_data)} customers")
    print(f" Employee Performance: {len(employee_data)} employees")
    
    
    print("\n Sample Sales Data:")
    print(sales_data.head())

In [ ]:
if conn:
    print(" KEY BUSINESS METRICS")
    print("=" * 50)
    
    
    total_revenue = sales_data['total_revenue'].sum()
    print(f" Total Revenue: ${total_revenue:,.2f}")
    
    
    total_orders = sales_data['total_orders'].sum()
    print(f" Total Orders: {total_orders:,}")
    
    
    unique_customers = len(customer_data)
    print(f" Active Customers: {unique_customers:,}")
    
   
    avg_order_value = sales_data['avg_order_value'].mean()
    print(f" Average Order Value: ${avg_order_value:.2f}")
    
    
    top_product = product_data.iloc[0]['ProductName']
    top_product_revenue = product_data.iloc[0]['total_revenue']
    print(f" Top Product: {top_product} (${top_product_revenue:,.2f})")
    
    
    top_customer = customer_data.iloc[0]['CompanyName']
    top_customer_revenue = customer_data.iloc[0]['total_revenue']
    print(f" Top Customer: {top_customer} (${top_customer_revenue:,.2f})")


In [ ]:
if conn:
    print("📊 Creating Revenue Trend Analysis...")
    
    
    fig1 = px.line(sales_data, x='order_month', y='total_revenue', 
                   title='Monthly Revenue Trend',
                   labels={'total_revenue': 'Revenue ($)', 'order_month': 'Month'})
    fig1.update_layout(height=500, showlegend=False)
    fig1.show()
    
   
    fig2 = px.bar(sales_data, x='order_month', y='revenue_growth_rate', 
                  title='Month-over-Month Revenue Growth Rate (%)',
                  labels={'revenue_growth_rate': 'Growth Rate (%)', 'order_month': 'Month'})
    fig2.update_layout(height=500)
    fig2.show()
    
    
    fig3 = px.area(sales_data, x='order_month', y='cumulative_revenue', 
                   title='Cumulative Revenue Over Time',
                   labels={'cumulative_revenue': 'Cumulative Revenue ($)', 'order_month': 'Month'})
    fig3.update_layout(height=500)
    fig3.show()

In [ ]:
if conn:
    print("🛍️ Creating Product Performance Analysis...")

    
    
    top_15_products = product_data.head(15)
    fig4 = px.bar(top_15_products, x='ProductName', y='total_revenue', 
                  color='CategoryName',
                  title='Top 15 Products by Revenue')
    fig4.update_xaxes(tickangle=45)
    fig4.update_layout(height=600)
    fig4.show()
    
    
    category_revenue = product_data.groupby('CategoryName')['total_revenue'].sum().reset_index()
    fig5 = px.pie(category_revenue, values='total_revenue', names='CategoryName', 
                  title='Revenue Distribution by Category')
    fig5.update_layout(height=500)
    fig5.show()
    
    
    category_quantity = product_data.groupby('CategoryName')['total_quantity_sold'].sum().reset_index()
    fig6 = px.bar(category_quantity, x='CategoryName', y='total_quantity_sold', 
                  title='Total Quantity Sold by Category')
    fig6.update_layout(height=500)
    fig6.show()

In [ ]:
if conn:
    print("Creating Customer Analysis...")
    
    
    value_segments = customer_data['value_segment'].value_counts()
    fig7 = px.pie(values=value_segments.values, names=value_segments.index, 
                  title='Customer Value Segments')
    fig7.update_layout(height=500)
    fig7.show()
    
    
    freq_segments = customer_data['frequency_segment'].value_counts()
    fig8 = px.pie(values=freq_segments.values, names=freq_segments.index, 
                  title='Customer Purchase Frequency')
    fig8.update_layout(height=500)
    fig8.show()
    
    
    top_20_customers = customer_data.head(20)
    fig9 = px.bar(top_20_customers, x='CompanyName', y='total_revenue', 
                  title='Top 20 Customers by Revenue')
    fig9.update_xaxes(tickangle=45)
    fig9.update_layout(height=600)
    fig9.show()
    
   
    fig10 = px.histogram(customer_data, x='total_revenue', nbins=30, 
                         title='Customer Revenue Distribution')
    fig10.update_layout(height=500)
    fig10.show()


In [ ]:
if conn:
    print("Creating Employee Performance Analysis...")
    
    
    fig11 = px.bar(employee_data, x='EmployeeName', y='total_sales', 
                   title='Sales Performance by Employee')
    fig11.update_xaxes(tickangle=45)
    fig11.update_layout(height=500)
    fig11.show()
    
    
    fig12 = px.scatter(employee_data, x='orders_handled', y='total_sales', 
                       size='unique_customers_served', hover_name='EmployeeName',
                       title='Orders Handled vs Total Sales by Employee')
    fig12.update_layout(height=500)
    fig12.show()
    
    
    fig13 = px.bar(employee_data, x='EmployeeName', y='avg_order_value', 
                   title='Average Order Value by Employee')
    fig13.update_xaxes(tickangle=45)
    fig13.update_layout(height=500)
    fig13.show()


In [ ]:
if conn:
    print(" Creating Geographic Analysis...")
    
    
    country_revenue = customer_data.groupby('Country').agg({
        'total_revenue': 'sum',
        'order_count': 'sum'
    }).reset_index().sort_values('total_revenue', ascending=False).head(10)
    
    fig14 = px.bar(country_revenue, x='Country', y='total_revenue', 
                   title='Top 10 Countries by Revenue')
    fig14.update_xaxes(tickangle=45)
    fig14.update_layout(height=500)
    fig14.show()
    
   
    fig15 = px.scatter(country_revenue, x='order_count', y='total_revenue', 
                       hover_name='Country', size='total_revenue',
                       title='Revenue vs Order Count by Country')
    fig15.update_layout(height=500)
    fig15.show()


In [ ]:
if conn:
    print("📊 Creating Advanced Analytics - Cohort Analysis...")
    
    
    customer_data['first_order_month'] = pd.to_datetime(customer_data['first_order_date']).dt.to_period('M')
    customer_acquisition = customer_data.groupby('first_order_month').size().reset_index(name='new_customers')
    customer_acquisition['first_order_month'] = customer_acquisition['first_order_month'].astype(str)
    
    fig16 = px.line(customer_acquisition, x='first_order_month', y='new_customers', 
                    title='Customer Acquisition by Month')
    fig16.update_xaxes(tickangle=45)
    fig16.update_layout(height=500)
    fig16.show()
    
    
    fig17 = px.box(customer_data, y='total_revenue', x='value_segment',
                   title='Customer Lifetime Value Distribution by Segment')
    fig17.update_layout(height=500)
    fig17.show()

In [ ]:
if conn:
    print(" DASHBOARD SUMMARY")
    print("=" * 60)
    
    
    metrics_summary = {
        'Metric': [
            'Total Revenue',
            'Total Orders', 
            'Active Customers',
            'Average Order Value',
            'Top Product Revenue',
            'Top Customer Revenue',
            'Best Performing Employee',
            'Top Country Revenue'
        ],
        'Value': [
            f"${total_revenue:,.2f}",
            f"{total_orders:,}",
            f"{unique_customers:,}",
            f"${avg_order_value:.2f}",
            f"${top_product_revenue:,.2f}",
            f"${top_customer_revenue:,.2f}",
            f"{employee_data.iloc[0]['EmployeeName']} (${employee_data.iloc[0]['total_sales']:,.2f})",
            f"{country_revenue.iloc[0]['Country']} (${country_revenue.iloc[0]['total_revenue']:,.2f})"
        ]
    }
    
    summary_df = pd.DataFrame(metrics_summary)
    print(summary_df.to_string(index=False))
    
    
    print("\n KEY BUSINESS INSIGHTS:")
    print("-" * 60)
    
   
    latest_growth = sales_data['revenue_growth_rate'].iloc[-1]
    if pd.notna(latest_growth):
        print(f"• Latest month revenue growth: {latest_growth:.1f}%")
    
    
    high_value_customers = len(customer_data[customer_data['value_segment'] == 'High Value'])
    high_value_percentage = (high_value_customers / len(customer_data)) * 100
    print(f"• High-value customers: {high_value_customers} ({high_value_percentage:.1f}%)")
    
    
    top_category = category_revenue.iloc[0]['CategoryName']
    top_category_revenue = category_revenue.iloc[0]['total_revenue']
    print(f"• Top performing category: {top_category} (${top_category_revenue:,.2f})")
    
    
    top_employee = employee_data.iloc[0]['EmployeeName']
    top_employee_sales = employee_data.iloc[0]['total_sales']
    print(f"• Top sales employee: {top_employee} (${top_employee_sales:,.2f})")


In [ ]:
if conn:
 
    sales_data.to_csv('sales_overview.csv', index=False)
    product_data.to_csv('product_performance.csv', index=False)
    customer_data.to_csv('customer_analysis.csv', index=False)
    employee_data.to_csv('employee_performance.csv', index=False)
    

In [ ]:
dashboard_code = '''
import streamlit as st
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

st.set_page_config(
    page_title="Northwind Sales Dashboard",
    layout="wide"
)


@st.cache_data
def load_data():
    conn = sqlite3.connect('northwind.db')
    
    sales_data = pd.read_csv('sales_overview.csv')
    product_data = pd.read_csv('product_performance.csv')
    customer_data = pd.read_csv('customer_analysis.csv')
    employee_data = pd.read_csv('employee_performance.csv')
    
    return sales_data, product_data, customer_data, employee_data


def main():
    st.title(" Northwind Sales Performance Dashboard")
    
    
    sales_data, product_data, customer_data, employee_data = load_data()
    
   
    st.sidebar.title("Dashboard Navigation")
    page = st.sidebar.selectbox("Select Page", 
                               ["Executive Summary", "Sales Analysis", 
                                "Product Performance", "Customer Analysis"])
    
    if page == "Executive Summary":
        st.header("Executive Summary")
        col1, col2, col3, col4 = st.columns(4)
        with col1:
            st.metric("Total Revenue", f"${sales_data['total_revenue'].sum():,.2f}")
        with col2:
            st.metric("Total Orders", f"{sales_data['total_orders'].sum():,}")
        with col3:
            st.metric("Active Customers", f"{len(customer_data):,}")
        with col4:
            st.metric("Avg Order Value", f"${sales_data['avg_order_value'].mean():.2f}")
        
        fig = px.line(sales_data, x='order_month', y='total_revenue', 
                      title='Monthly Revenue Trend')
        st.plotly_chart(fig, use_container_width=True)
        
    elif page == "Sales Analysis":
        st.header("Sales Analysis")
        st.write("This page will contain detailed sales analysis charts and tables.")
        fig = px.bar(sales_data, x='order_month', y='total_orders', title='Monthly Orders')
        st.plotly_chart(fig, use_container_width=True)
    
    elif page == "Product Performance":
        st.header("Product Performance")
        # st.write("This page will show insights on product performance.")
        top_products = product_data.sort_values('total_revenue', ascending=False).head(10)
        fig = px.bar(top_products, x='ProductName', y='total_revenue', title='Top 10 Products by revenue')
        st.plotly_chart(fig, use_container_width=True)
    
    elif page == "Customer Analysis":
        st.header("Customer Analysis")
        # st.write("This page will provide analysis on customers.")
        if 'value_segment' in customer_data.columns:
            segment_counts = customer_data['value_segment'].value_counts()
            fig = px.pie(values=segment_counts.values, names=segment_counts.index, title='Customer Value Segments')
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.write("Customer value segment data not available.")



if __name__ == "__main__":
    main()
'''

with open('streamlit_dashboard.py', 'w') as f:
    f.write(dashboard_code)